In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn as sk 
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [2]:
raw_data = pd.read_excel("Data_2023.xlsx")

In [3]:
Dates = []
for i in range(0, 19, 4):
    Dates.append(raw_data.iloc[:, i:i+4])



In [4]:
for i in range (len(Dates)):
    Dates[i] = Dates[i].rename(columns = {Dates[i].columns[0]:"VV"})
    Dates[i] = Dates[i].rename(columns = {Dates[i].columns[1]:"VH"})
    Dates[i] = Dates[i].rename(columns = {Dates[i].columns[2]:"SM"})
    Dates[i] = Dates[i].rename(columns = {Dates[i].columns[3]:"LAI"})


In [5]:
Merged_Data = pd.concat(Dates, ignore_index=True)

In [6]:
x_data = Merged_Data.iloc[:, 0:2]
y_data = Merged_Data.iloc[:, 2:]

In [7]:
x_train, x_test, y_train, y_test  = train_test_split(x_data, y_data, test_size = 0.2, train_size= 0.8)

In [8]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
x_train = min_max_scaler.fit_transform(x_train)
x_test = min_max_scaler.transform(x_test)

min_max_scaler_y = MinMaxScaler(feature_range=(-1,1))
y_train = min_max_scaler_y.fit_transform(y_train)


In [9]:
def plot_predictions(y_test, predictions, title):
    plt.scatter(y_test, predictions, alpha=0.5)
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.xlim(0, 7)
    plt.ylim(0, 7)
    plt.title(title)
    plt.show()

In [10]:
Ridge_models = []
for i in range(0, 1000):
   Ridge_models.append(linear_model.Ridge(alpha=i/1000))
   Ridge_models[-1].fit(x_train,y_train)
   

In [11]:
Ridge_mod_pred = []
for i in Ridge_models:
    Ridge_mod_pred.append(i.predict(x_test))
for i, v in enumerate (Ridge_mod_pred):
   Ridge_mod_pred[i] = min_max_scaler_y.inverse_transform(v)

y_test = pd.DataFrame(y_test)
Ridge_Error_sm = []
Ridge_Error_lai = []
Ridge_Error_r2 = []
for i in Ridge_mod_pred:
    i = pd.DataFrame(i)
    Ridge_Error_sm.append((root_mean_squared_error(y_test.iloc[:, 0], i.iloc[:, 0])))
    Ridge_Error_lai.append((root_mean_squared_error(y_test.iloc[:, 1], i.iloc[:, 1])))
    Ridge_Error_r2.append((r2_score(y_test.iloc[:, 1], i.iloc[:, 1])))
min_error_sm_ridge = min(Ridge_Error_sm)
min_error_lai_ridge = min(Ridge_Error_lai)
max_r2 = max(Ridge_Error_r2)
print([min_error_sm_ridge, min_error_lai_ridge])
print(max_r2)


[0.11698624398548654, 1.5275741442718183]
0.048395708518914504


In [12]:

svr_sm_linear = svm.SVR(kernel="linear")
svr_sm_linear.fit(x_train, y_train[:, 0])
svr_lai_linear = svm.SVR(kernel="linear")
svr_lai_linear.fit(x_train, y_train[:, 1])



SVR(kernel='linear')

In [13]:
svr_pred_sm_linear = svr_sm_linear.predict(x_test)
svr_pred_lai_linear = svr_lai_linear.predict(x_test)

svr_pred_linear = pd.DataFrame({0: svr_pred_sm_linear, 1:svr_pred_lai_linear})
svr_pred_linear = min_max_scaler_y.inverse_transform(svr_pred_linear)



svr_error_linear_sm = root_mean_squared_error(svr_pred_linear[:, 0], y_test.iloc[:, 0])
svr_error_linear_lai = root_mean_squared_error(svr_pred_linear[:, 1], y_test.iloc[:, 1])
svr_r2_linear_sm = r2_score(svr_pred_linear[:, 0], y_test.iloc[:, 0])
svr_r2_linear_lai = r2_score(svr_pred_linear[:, 1], y_test.iloc[:, 1])


print([svr_error_linear_sm,svr_error_linear_lai])
print([svr_r2_linear_sm, svr_r2_linear_lai])


[0.11776237760571892, 1.5079330661538919]
[-20.73179120205367, -8.762659766336508]


In [14]:
svr_sm_rbf = svm.SVR(kernel="rbf")
svr_sm_rbf.fit(x_train, y_train[:, 0])
svr_lai_rbf = svm.SVR(kernel="rbf")
svr_lai_rbf.fit(x_train, y_train[:, 1])

svr_pred_sm_rbf = svr_sm_rbf.predict(x_test)
svr_pred_lai_rbf = svr_lai_rbf.predict(x_test)

svr_pred_rbf = pd.DataFrame({0: svr_pred_sm_rbf, 1:svr_pred_lai_rbf})

svr_pred_rbf = min_max_scaler_y.inverse_transform(svr_pred_rbf)

svr_pred_sm_rbf_error = root_mean_squared_error(svr_pred_rbf[:, 0], y_test.iloc[:, 0])
svr_pred_lai_rbf_error = root_mean_squared_error(svr_pred_rbf[:, 1], y_test.iloc[:, 1])
svr_pred_sm_rbf_r2 = r2_score(svr_pred_rbf[:, 0], y_test.iloc[:, 0])
svr_pred_lai_rbf_r2 = r2_score(svr_pred_rbf[:, 1], y_test.iloc[:, 1])
print([svr_pred_sm_rbf_error,svr_pred_lai_rbf_error])
print([svr_pred_sm_rbf_r2, svr_pred_lai_rbf_r2])


[0.11714191495756597, 1.5757988477371045]
[-19.814993988085032, -3.247041415959414]


In [15]:



gbr_sm = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, max_depth=2, random_state=42, min_samples_split=10, min_samples_leaf=5, subsample=0.8)
gbr_sm.fit(x_train, y_train[:, 0])
gbr_sm_pred = gbr_sm.predict(x_test)


gbr_lai = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, max_depth=2, random_state=42, min_samples_split=10, min_samples_leaf=5, subsample=0.8)
gbr_lai.fit(x_train, y_train[:, 1])
gbr_lai_pred = gbr_lai.predict(x_test)


gbr_pred = pd.DataFrame({0: gbr_sm_pred, 1: gbr_lai_pred})
gbr_pred = min_max_scaler_y.inverse_transform(gbr_pred)


gbr_error_sm = root_mean_squared_error(gbr_pred[:, 0], y_test.iloc[:, 0])
gbr_error_lai = root_mean_squared_error(gbr_pred[:, 1], y_test.iloc[:, 1])
gbr_r2_sm = r2_score(gbr_pred[:, 0], y_test.iloc[:, 0])
gbr_r2_lai = r2_score(gbr_pred[:, 1], y_test.iloc[:, 1])

cv_scores = cross_val_score(GradientBoostingRegressor(random_state=42), x_train, y_train[:, 0], cv=5, scoring='r2')
print("Cross-validated R2 scores:", cv_scores)

print([gbr_error_sm, gbr_error_lai])
print([gbr_r2_sm, gbr_r2_lai])



Cross-validated R2 scores: [-0.05146874 -0.55909104 -1.02118865 -0.97229483 -0.08363708]
[0.11697707436133771, 1.536360867382797]
[-8.659675208184758, -4.292512449334922]
